# Process Board of Education Motions

Process the motions scraped from the Shelby County Board of Education minutes, 
get a better understanding of them, and make a nicely-formatted table for delivery.

In [1]:
import re
import pandas as pd

In [2]:
motions = pd.read_csv('boe-motions.csv')

In [3]:
# Extract sections of the motions
motions['summary'] = motions.motion.str.extract(
    "(It [is ]{0,3}recommende?d [\s\S]*)\n\n?Motion by",
    flags=re.IGNORECASE).fillna('')
motions['summary'] = motions['summary'].str.strip('\n')
motions['result'] = motions.motion.str.extract("Final Resolution: ([\w ]*)",
                                               flags=re.IGNORECASE)

motions['sponsor'] = motions.motion.str.extract("Motion by ([\w ]*)",
                                                flags=re.IGNORECASE)
motions['second'] = motions.motion.str.extract("second by ([\w ]*)",
                                               flags=re.IGNORECASE)

for vote in ('Affirmative', 'Negative', 'Not Present at Vote', 'Abstain'):
    motions[vote.lower().replace(' ', '_')] = motions.motion.str.extract(
        f"{vote}: ([\w ,]*)", flags=re.IGNORECASE).fillna('')


In [4]:
motions.date = pd.to_datetime(motions.date.str[:-6])
motions.sort_values(by='date', inplace=True)

In [5]:
# Michelle McKissack is the mayoral candidate on this board
with_michelle = (motions.affirmative + motions.negative +
                 motions.abstain).str.contains('McKissack', case=False)
with_michelle.value_counts()

True     1562
False     449
dtype: int64

In [6]:
# Only 39 of 2011, or ~2% of motions are contested
contested = (motions.affirmative != '') & (motions.negative != '')
motions['contested'] = contested
contested.value_counts()

False    1972
True       39
dtype: int64

In [7]:
# Michelle never abstained
motions.abstain.str.contains('McKissack', case=False).value_counts()

False    2011
Name: abstain, dtype: int64

In [8]:
# How did Michelle vote?

motions.loc[with_michelle, 'mckissack'] = [
    'for' if 'McKissack' in row.affirmative else 'against'
    for _, row in motions[with_michelle].iterrows()
]
motions['mckissack'].fillna('', inplace=True)
motions[with_michelle].affirmative.str.contains('McKissack',
                                                case=False).value_counts()

True     1541
False      21
Name: affirmative, dtype: int64

In [9]:
motions[contested & with_michelle].mckissack.value_counts()

for        20
against    11
Name: mckissack, dtype: int64

In [10]:
motions.head()

,date,meeting_title,action,motion,summary,result,sponsor,second,affirmative,negative,not_present_at_vote,abstain,contested,mckissack
327,2018-01-30,January Board Business Meeting,11.2 Medicaid Reimbursement,It is recommended that the Shelby County Board...,It is recommended that the Shelby County Board...,Motion Passes,Shante Avant,Stephanie P Love,"Shante Avant, Miska Clay Bibbs, Chris Caldwell...",,Billy Orgel,,False,
323,2018-01-30,January Board Business Meeting,"7.1 Approval of the Minutes of the December 5,...",It is recommended that the Shelby County Board...,It is recommended that the Shelby County Board...,Motion Passes,Scott McCormick,Chris Caldwell,"Shante Avant, Miska Clay Bibbs, Chris Caldwell...",,Stephanie P Love,,False,
324,2018-01-30,January Board Business Meeting,"7.2 Approval of the Minutes of the January 23,...",It is recommended that the Shelby County Board...,It is recommended that the Shelby County Board...,Motion Passes,Scott McCormick,Chris Caldwell,"Shante Avant, Miska Clay Bibbs, Chris Caldwell...",,Stephanie P Love,,False,
325,2018-01-30,January Board Business Meeting,7.3 Approval of the Minutes of the December 13...,It is recommended that the Shelby County Board...,It is recommended that the Shelby County Board...,Motion Passes,Scott McCormick,Chris Caldwell,"Shante Avant, Miska Clay Bibbs, Chris Caldwell...",,Stephanie P Love,,False,
326,2018-01-30,January Board Business Meeting,11.1 Approval of Consent Agenda,It is recommended that the Shelby County Board...,It is recommended that the Shelby County Board...,Motion Passes,Shante Avant,Stephanie P Love,"Shante Avant, Miska Clay Bibbs, Chris Caldwell...",,Billy Orgel,,False,


In [11]:
motions.to_csv('boe-motions-processed.csv', index=False)